In [ ]:
import Logging: global_logger
import TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

In [ ]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using FFTW
using Plots

In [ ]:
xleft::Float64 = 0.0
xright::Float64 = 1.0
t0::Float64 = 0.0
T::Float64 = 20.0
N = 100
order = 2
step = (xright - xleft)/N
grid = collect(xleft:step:xright)

#Definition of constants (placeholders):
gamma::Float64 = 0.25
gamma_3::Float64 = 0.5
c::Float64 = 3
omega_d::Float64 = 10.0
A_forcing::Float64 = 0.3
lambda_forcing::Float64 = -40

#Define parameters and variables (note: u(..) is valid as a symbolic function notation):
@parameters x t
@variables u(..)

#Define differential operators:
Dt = Differential(t)
Dx = Differential(x)
Dtt = Differential(t)^2
Dxx = Differential(x)^2

#Define source function (placeholder constant spatial term):
source_f(x, t) = A_forcing * exp(lambda_forcing*(x)^2) * sin(omega_d*t)

#Define initial condition (placeholder sinusoidal):
#u0(x, t) = exp(-(35*pi)*(x-0.5)^2)
u0(x, t) = 0
v0(x) = 0


#Define equation (with and without cubic damping term):
equation = [Dtt(u(x,t)) + gamma*Dt(u(x,t)) + gamma_3*(Dt(u(x,t)))^3 ~ c^2*Dxx(u(x,t))]
#equation = [Dtt(u(x,t)) ~ c^2*Dxx(u(x,t))]


#Define domains:
domains = [x ∈ Interval(xleft, xright), t ∈ Interval(t0, T)]

#Define boundary conditions:
bound_conditions = [u(x, 0) ~ u0(x, 0),
                     u(xleft, t) ~ A_forcing * sin(omega_d*t),
                     Dx(u(xright, t)) ~ 0,
                     Dt(u(x, 0)) ~ v0(x)]

In [ ]:
#Create PDE system and assign identifier (order matters: eq, bcs, domains, paramers, variables; format parameters and variables as lists, provide arguments whenever they exist):
@named pde_sys = PDESystem(equation, bound_conditions, domains, [x, t], [u(x,t)])

#Define a discretization (first argument specifies parameters to discretize, second indicates parameters to keep continuous, third provides the approximation order):
discretization = MOLFiniteDifference([x=>grid], t, approx_order=order)

#Proceed with discretization:
@time eq_discretized = discretize(pde_sys,discretization)

#Solve the resulting system (specified solver dictates whether or not time marching is used, as well as the method: Euler(), ImplicitEuler(), or Trapezoidal(), all with given time steps):
#Warning: if the time step is not explicitly given, solver may default to actual method of lines, instead of time marching\
dt = 1e-2
solution= solve(eq_discretized, ImplicitEuler(), dt=dt, saveat=0.01, progress=true, progress_steps=50)

In [ ]:

# Your solution data
discrete_t = solution.t
sol_u = solution[u(x, t)]
grid = grid

Nx = length(grid)
Nt = length(discrete_t)

# Create the animation object
anim = @animate for k in 1:Nt
    t = discrete_t[k]
    plot(
        grid, sol_u[:, k],
        ylim = (-1.0, 1.0),
        xlabel = "x",
        ylabel = "u(x, t)",
        title = "t = $(round(t, digits=2))",
        legend = false
    )
end

# Save the animation as a GIF
gif(anim, "solution_animation_1.gif", fps = 60)


In [ ]:
# Fourier Transform analysis
using Plots
using FFTW

gr()  # or pyplot()

k = 1500
fs = N   # sampling frequency

u_at_k = sol_u[:, k]

F = fftshift(fft(u_at_k))
freqs = fftshift(fftfreq(length(u_at_k), fs))

# First subplot: u(x)
p1 = plot(
    grid,
    u_at_k,
    xlabel = "Distance (m)",
    ylabel = "Displacement (m)",
    legend = false
)

# Second subplot: Fourier spectrum
p2 = plot(
    freqs,
    abs.(F) ./ fs,
    xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
    xlim = (-50, 50),
    legend = false
)

# Combine plots side by side
plot(p1, p2, layout = (1, 2), size = (900, 400))

In [ ]:
using Plots
using FFTW

u_at_point = sol_u[36, :]
fs_time = Nt / T   # sampling frequency in time

F = fftshift(fft(u_at_point))
freqs = fftshift(fftfreq(length(u_at_point), fs_time))

plot(
    freqs,
    abs.(F) ./ fs_time,
    xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
    xlim = (-50, 50),
    legend = false,
    size = (500, 400)
)

###### 